# Problema modelado a objetos
El problema que seleccione es dada una cuadricula con hoyos, un punto de partida y punto que llamaremos final, encontrar el mejor camino que los una.

Para la orientación a objetos decidi tener 3 clases:
* Punto: Hace referencia a una casilla en la cuadricula.
* Camino: Hace referencia a un conjunto de Puntos con ciertas restricciones, como que los puntos sean continuos y además que describan un camino en la cuadricula.
* Plano: Hace referencia a un conjunto de puntos, donde hay un punto inicio y un punto final.

Las relaciones entre estos objetos se ven abajo.

## Definición de clases

In [63]:
import random

class Punto:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __eq__(self, B):
        if  isinstance(B, Punto):
            return (self.x == B.x) and (self.y == B.y)

        if isinstance(B, Camino):
            if len(B) == 1:
                return self == B.camino[0] 
            else:
                return False
                
        return False
        
    def __hash__(self):
        return hash((self.x, self.y))

    def arriba(self):
        '''
            Regresa unto con cordenadas (x, y+1), el cual corresponde al punto de arriba
        '''
        return Punto(self.x, self.y + 1)
        
    def abajo(self):
        '''
            Regresa unto con cordenadas (x, y-1), el cual corresponde al punto de abajo
        '''
        return Punto(self.x, self.y - 1)
        
    def derecha(self):
        '''
            Regresa unto con cordenadas (x+1, y), el cual corresponde al punto de la derecha
        '''
        return Punto(self.x + 1, self.y)
        
    def izquierda(self):
        '''
            Regresa unto con cordenadas (x-1, y), el cual corresponde al punto de la izquierda
        '''
        return Punto(self.x - 1, self.y)
        
    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
        
    def __repr__(self):
        return str(self)
        
    def __sub__(self, A):
        assert isinstance(A, Punto), "No es posible restar"
        return Punto(self.x - A.x, self.y - A.y)
        
    def distancia(self, B):
        '''
            Distancia L1 entre los puntos
        '''
        assert isinstance(B, Punto), "Debe ser punto"
        return abs(self.x - B.x) + abs(self.y - B.y)

In [71]:
class Camino:
    def __init__(self, entrada):
        #Se puede iniciar con un punto o una lista de puntos continuos
        espunto = isinstance(entrada, Punto)
        
        eslista = isinstance(entrada, list)
        
        eslistcamino = True
        
        if eslista:
            #revizamos si todos son puntos
            eslistcamino = all([isinstance(punto, Punto) for punto in entrada])
            
            #ahora si estan a distancia uno
            for i in range(len(entrada)-1):
                eslistcamino = eslistapuntos and (Punto.distancia(entrada[i], entrada[i+1]) == 1)
                
        assert  espunto or (eslista and eslistcamino), "debe ser lista continua o punto"

        
        if espunto:
            self.camino = [entrada]
        else:
            self.camino = list(entrada)
            
    def __str__(self):
        return "".join(self.strCamino())
        
    def __repr__(self):
        return str(self)
        
    def agregarPunto(self, A):
        '''
            Agrega un punto al final del camino
        '''
        assert isinstance(A, Punto), "Solo puedes agregar puntos"
        assert self.camino[-1].distancia(A) == 1, "El punto a agregar debe estar pegado al final del camino"
        
        self.camino.append(A)

    def getPuntos(self):
        '''
            Regresa una copia de los puntos de la instancia
        '''
        return list(self.camino)
        
    def __contains__(self, A):
        assert isinstance(A, Punto), 'Debe ser un punto'
        return A in self.getPuntos()

    def strCamino(self):
        """
        regresa una lista con strings de felchas indicando en que direccion fue cada movimiento del camino
        """
        if len(self.camino) == 1:
            return ["•"]
        caminos = []

        #la primera flecha es distinta
        if ((self.camino[1] - self.camino[0]) == Punto(1, 0)):
            caminos.append("↦")
        elif ((self.camino[1] - self.camino[0]) == Punto(0, 1)):
            caminos.append("↧")
        elif ((self.camino[1] - self.camino[0]) == Punto(-1, 0)):
            caminos.append("↤")
        else:
            caminos.append("↥")
            
        for i in range(1,len(self.camino)-1):
            if ((self.camino[i + 1] - self.camino[i]) == Punto(1, 0)):
                caminos.append("→")
            elif ((self.camino[i + 1] - self.camino[i]) == Punto(0, 1)):
                caminos.append("↓")
            elif ((self.camino[i + 1] - self.camino[i]) == Punto(-1, 0)):
                caminos.append("←")
            else:
                caminos.append("↑")
                
        caminos.append("•")
        return caminos
        
    def __len__(self):
        '''
            regresa numero de puntos en el camino
        '''
        return len(self.camino)
        
    def __getitem__(self, key):
        esslice = isinstance(key, slice)
        esint = isinstance(key, int)
        assert esslice or esint, "error" 

        
        return Camino(self.camino[key])

        

In [72]:
class Plano:
    def __init__(self, x, y, inicio = None, final = None, huecos = set()):
        #Agregar asserts      
        self.lenx = x
        self.leny = y
        self.puntos = [[Punto(x1,y1) for x1 in range(int(x)+1)]  for y1 in range(int(y+1))]       
        self.huecos = [hueco for hueco in huecos if hueco in self]

        for hueco in self.huecos:
            self.puntos[hueco.y][hueco.x] = None
        
        #cambiar nombre
        
        # lista = self.getPuntos()
        # lista.remove(self.inicio)

        if not (inicio is None):
            assert inicio in self, "inicio no esta en el plano"
            self.inicio = inicio 
        else:
           self.inicio = random.choice(self.getPuntos())
            
        if not(final is None):
            assert final in self, "final no esta en el plano"
            self.final = final 
        else:
            self.final = random.choice(lista) 
        
 
        
    def distancia(self, A, B):
        '''
            Te da el numero de pasos necesarios para llegar a A a B en el plano.
        '''
        #Falta considerar hoyos
        assert isinstance(A, Punto) and isinstance(B, punto), "entradas deben ser de tipo punto"    
        return abs(A.x-B.x) + abs(A.y-B.y)

    def puntosCercanos(self, A): 
        '''
            Regresa lista con puntos adjacentes al punto A que esten en el plano.
        '''
        assert A in self, "Este punto no esta en el plano"
        puntosCercanos = [] #cambiar a set?
        puntosAdjacentos = {A.arriba(), A.abajo(), A.derecha(),A.izquierda}
        
        for punto in puntosAdjacentos:
            if punto in self:
                puntosCercanos.append(punto)
                
        return puntosCercanos
  
    def getPuntos(self):
        '''
            Te regresa los puntos 
        '''
        puntos = []
        for lista_puntos in self.puntos:
            for punto in lista_puntos:
                puntos.append(punto)
        return puntos

    def __contains__(self, A):
        '''
            Verifica si el camino o punto esta en el plano
        '''
        if isinstance(A, Punto):          
            return A in self.getPuntos()
            
        elif isinstance(A, Camino):
            return all([(B in self) for B in A.getPuntos()])
            
        return False

    def __str__(self):
        '''
            Representacion vizual del plano
        '''
        string = "   " + " ".join([str(n).ljust(2, " ") for n in range(self.lenx + 1)]) + "\n"
        for y in range(self.leny + 1): 
            string = (string + str(y)) if len(str(y)) == 2 else (string + " "+ str(y))
            for x in range(self.lenx + 1):
                
                if not (self.puntos[y][x] is None):
                    string += " * "
                else:
                    string += " x "
            string = string + "\n" 
        return string
        
    def __repr__(self):
        return str(self)

    def __getitem__(self, indices):
        '''
            Te regresa un subplano con mismos huecos pero con final e inicio distinto
        '''
        
        return Plano(indices[0], indices[1], huecos = self.huecos)

    def hacerHueco(self, A):
        '''
            Agrega un hueco en el punto dado
        '''
        assert isinstance(A, Punto), "No es un punto"
        self.puntos[A.y][A.x] = None

    def solucionAleatoria(self, camino = None):
        '''
            Moviendose a partir del inicio y tomando deciciones aleatorias te regresa el camino descrito.
            No necesariamente llega al final.
        '''
        if camino is None:
            camino = Camino(self.inicio)
        #lista de puntos a distancia 1
        puntosQuePuedenSerOpciones = [camino.getPuntos()[-1].arriba(),
                                      camino.getPuntos()[-1].abajo(), 
                                      camino.getPuntos()[-1].izquierda(), 
                                      camino.getPuntos()[-1].derecha()]
        
        #revizamos cuales estan en el plano
        
        puntosPosibles = [punto for punto in puntosQuePuedenSerOpciones if ((punto in self) and not(punto in camino))]
        
        if puntosPosibles:
            
            #seleccionamos uno
            puntoNuevo = random.choice(puntosPosibles)
            
            camino.agregarPunto(puntoNuevo)
            return camino if self.final == camino.getPuntos()[-1] else self.solucionAleatoria(camino = camino)
            
        return camino
        
    def printCamino(self, camino):
        '''
            Dado un camino te regresa su representacion vizual en el plano
        '''
        string = "   " + " ".join([str(n).ljust(2, " ") for n in range(self.lenx + 1)]) + "\n"
        simbolos = camino.strCamino()
        
        for y in range(self.leny + 1 ):
            string = (string + str(y)) if len(str(y)) == 2 else (string + " "+ str(y)) 
            for x in range(self.lenx + 1):
                if not (self.puntos[y][x] is None):
                    if self.puntos[y][x] in camino:
                        string += " " + simbolos[(camino.getPuntos()).index(self.puntos[y][x])] + " "
                    else:
                        string += " * "
                else:
                    string += " x "
            string += "\n"
        print(string)
        
    def solucionesAleatorias(self, size):
        '''
          te regresa una lista con size soluciones usando el metodo solucionAleatoria
        '''
        assert isinstance(size,int), "size debe ser entero"
        return [self.solucionAleatoria() for a in range(size)]
        
    def costo(self, camino):
        '''
            Dado un camino te regresa su costo, infinito si no llega al final y el numero de pasos si sí llega.
            Si le das una lista te regresa una lista con el costo de cada uno,
        '''
        escamino = isinstance(camino, Camino)
        eslista = isinstance(camino, list)
        eslistadepuntos = True
        
        if eslista:
            eslistadepuntos = all([isinstance(caminon, Camino) for caminon in camino])
        
        assert (escamino or (eslista and eslistadepuntos)), "Error"

        if escamino:
            assert camino in self, "Este camino no esta en el plano"
            if camino[-1] != self.final:
                return float("inf")
            return len(camino)

        return [self.costo(caminon) for caminon in camino]

    def solucionI(self, camino = None):
        """
            regresa una solucion que intenta llegar al final con un algoritmo muy malo pero funciona en casos sencillos
        """
        if camino is None:
            camino = Camino(self.inicio)

        if camino.getPuntos()[-1] == self.final:
            return camino
        #puntos cercan
        
        pcaminofinal =camino.getPuntos()[-1]
        
        parriba = pcaminofinal.arriba()
        pabajo = pcaminofinal.abajo()
        pderecha = pcaminofinal.derecha()
        pizquierda = pcaminofinal.izquierda()
        
        derechaoizquierdaposible = True
        algunofueposible = True
        
        #revizamos en que direccion esta el final
        
        derecha = self.final.x - pcaminofinal.x > 0#False if (pcaminofinal.x - self.final.x > 0) else True
        izquierda =  self.final.x - pcaminofinal.x < 0#False if (pcaminofinal.x - self.final.x < 0) else True 
        arriba = self.final.y - pcaminofinal.y > 0#False if (pcaminofinal.y - self.final.y > 0) else True
        abajo = self.final.y - pcaminofinal.y < 0#False if (pcaminofinal.y - self.final.y < 0) else True
        
        #Nos intentamos mover  a un lugar donde nos acerquemos al final
        if derecha:
            if (not (pderecha in camino)) and (pderecha in self):
                algunofueposible = False
                derechaoizquierdaposible = False
                #print("Se va a agregar derecha" + str(pderecha))
                camino.agregarPunto(pderecha)
                
        if izquierda:
            if (not (pizquierda in camino)) and (pizquierda in self):
                algunofueposible = False
                derechaoizquierdaposible = False
                #print("Se va a agregar izquierda" + str(pizquierda))
                camino.agregarPunto(pizquierda)

        #si no podemos vertical, vemos arriba o abajo
        if derechaoizquierdaposible:
            if arriba:
                if (not (parriba in camino)) and ( parriba in self):
                    algunofueposible = False
                    #print("Se va a agregar arriba" + str(parriba))
                    camino.agregarPunto(parriba)
            if abajo:
                if (not (pabajo in camino)) and (pabajo in self):
                    algunofueposible = False
                    #print("Se va a agregar abajo" + str(pabajo))
                    camino.agregarPunto(pabajo)
                
        #Si no nos pudimos mover a un lugar que nos acerque al final, tomamos uno que si se pueda aleatorio
        puntosQuePuedenSerOpciones = [parriba,
                                      pabajo, 
                                      pizquierda, 
                                      pderecha]
        
        #revizamos cuales estan en el plano
        puntosPosibles = [punto for punto in puntosQuePuedenSerOpciones if ((punto in self) and not(punto in camino))]

        
        if algunofueposible:
            
            #seleccionamos uno
            #print("Puntos posibles: " + str(puntosPosibles))
            if puntosPosibles:
                puntoNuevo = random.choice(puntosPosibles)
                 
                camino.agregarPunto(puntoNuevo)
                return self.solucionI(camino = camino)
            else:
                return camino
       
            
        return self.solucionI(camino = camino) 

## Busqueda de solución
Un ejemplo facil es un plano de (0,20) x (0,20) conlos huecos que se ven abajo y de inicio el (0,0) y final (20,20).

In [73]:
plano = Plano(20,20, 
              huecos = [Punto(15,15), Punto(15,0), Punto(20,15)],
              inicio = Punto(0,0), 
              final = Punto(20,20))


Ponemos en practica la solucion aleatoria y la solucionI

In [74]:
camino1 = plano.solucionAleatoria()

In [75]:
camino2 = plano.solucionI()

Vemos su costo

In [76]:
costo1 = plano.costo(camino1)
costo2 = plano.costo(camino2)
print('El costo de la solucion aleatoria es: ', costo1)
print('El costo de la solucion I es: ', costo2)

El costo de la solucion aleatoria es:  inf
El costo de la solucion I es:  43


Vemos la representacion vizual de las soluciones

In [77]:
# aleatoria
plano.printCamino(camino1)

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20
 0 ↧  *  *  *  *  *  *  *  *  *  *  *  *  *  *  x  *  *  *  *  * 
 1 →  →  ↓  →  →  →  ↓  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 2 →  •  ↓  ↑  ↓  ←  ←  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 3 ↑  ←  →  ↑  ↓  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 4 *  ↑  ←  ←  ←  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 5 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 6 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 7 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 8 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
 9 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
10 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
11 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
12 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
13 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
14 *  *  *

In [26]:
#solucion I
plano.printCamino(camino2)

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20
 0 ↦  →  →  →  →  →  →  →  →  →  →  →  →  →  ↓  x  *  *  *  *  * 
 1 *  *  *  *  *  *  *  *  *  *  *  *  *  *  →  →  →  →  →  →  ↓ 
 2 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 3 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 4 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 5 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 6 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 7 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 8 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
 9 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
10 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
11 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
12 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
13 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  ↓ 
14 *  *  *

# ¿Es posible resolver el problema que planteas analizando todas las posibles soluciones?
Supongamos que puedo dar todas las soluciones (no puedo), seria una cantidad demaciado grande que llevaria mucho tiempo.